In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import pandas as pd
import xarray as xr
import time
import seaborn as sns
from itertools import product
from scipy.interpolate import interp2d
import json
from glob import glob
import scipy.stats

from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction
from sklearn.gaussian_process import GaussianProcessRegressor 
from scipy.interpolate import griddata

In [ ]:
'''Load calibration streamflow data'''
dt2 =pd.read_csv('./rawdata/Streamflow_daily.csv', 
                 parse_dates=['Date'])

# Coerce the data into the types specified in the metadata  
dt2.Watershed = dt2.Watershed.astype('category') 

# Pull out 2017 year
stream = dt2[dt2.Date.dt.year == 2015]
stream = stream[stream.Watershed == 'S2'].reset_index(drop = True)

#Convert cm/day to mm/sec
m = 10/(60*60*24)
stream['Flow_mms'] = m*stream['Flow (cm/day)']

'''Load calibration WTE data'''
infile1  ="https://pasta.lternet.edu/package/data/eml/edi/562/2/671f15337a677da71852de506a8d9b05".strip() 
infile1  = infile1.replace("https://","http://")
                 
dt1 =pd.read_csv(infile1, skiprows = 1, sep = ",",
                 names=["PEATLAND", "DATE", "WTE", "FLAG"],
                 parse_dates=['DATE'], 
                 na_values={'WTE':['NA',], 'FLAG':['NA',]})

# Coerce the data into the types specified in the metadata  
dt1.PEATLAND = dt1.PEATLAND.astype('category') 

dt1.WTE = pd.to_numeric(dt1.WTE, errors ='coerce')  
dt1.FLAG = dt1.FLAG.astype('category') 

# Pull out 2017 year
wte =  dt1[dt1.DATE.dt.year == 2015]
wte = wte[wte.PEATLAND == 'S2'].reset_index(drop = True)
wte['WTD'] = -(422.0 - wte.WTE)

In [ ]:
'''Load SPRUCE and CLM5.0 data'''
SPRUCEuncalib = xr.open_mfdataset('/glade/u/home/marielj/peatlandMIP/ELM-SPRUCE/initRUN/TEST_US-SPR_ICB20TRCNPRDCTCBC/run/TEST_US-SPR_ICB20TRCNPRDCTCBC.clm2.h0.*-01-01-00000.nc', 
                            parallel = True)
CLMuncalib = xr.open_mfdataset('/glade/u/home/marielj/peatlandMIP/CLM5/initRUNS_long/mbp_tuning_spinup_grass_CONTROL_v0.clm2.h1.*-01-01-00000.nc', 
                            parallel = True)

In [ ]:
def runHillslope(slatop, leaflong, frootleaf, slopebeta, fff, baseflow, fmax, remove = False):
    #change parameter in netcdf file
    target_param_file = '/glade/work/marielj/inputdata/lnd/clm2/paramdata/clm5_params_augmented_base.c171117.nc'
    pft = 12 #arctic grass  
    
    #PFT Params
    change_pft_param('slatop', pft, slatop, target_param_file)
    change_pft_param('leaf_long', pft, leaflong, target_param_file)
    change_pft_param('froot_leaf', pft, frootleaf, target_param_file)
    
    #NON PFT Params
    change_param('slopebeta', slopebeta, target_param_file)
    change_param('fff', fff, target_param_file)    
    
    os.chdir(CASE_DIR)
    
    #Namelist Params
    change_nl_param('soil_fmax', fmax)
    change_nl_param('baseflow_scalar', baseflow)
    
    #run case
    pipe = subprocess.Popen(['qcmd', '-- ./case.submit'], stdout=subprocess.PIPE)
    #result = pipe.communicate()[0]
    #print(result)
    #print(CASE_NAME + " Run Complete")
    
    #time delay -- check if archived data exists, if not wait 5 more seconds
    SCRATCH_DIR = '/glade/scratch/marielj/archive/' + CASE_NAME + '/lnd/hist/'
    while(not os.path.exists(SCRATCH_DIR + CASE_NAME + '.clm2.h3.2017-01-01-00000.nc')):
        time.sleep(5)
    
    #find WTE data in scracth directory
    os.chdir(SCRATCH_DIR)
    
    #Open data h1 data file
    avgfiles = glob(SCRATCH_DIR + 'test-hillslope-mct-srof.clm2.h2.*.nc')
    colfiles = glob(SCRATCH_DIR + 'test-hillslope-mct-srof.clm2.h3.*.nc')
    
    avgdata = xr.open_mfdataset(avgfiles, parallel = True)
    coldata = xr.open_mfdataset(colfiles, parallel = True)
    
    #remove data
    if(remove):
        os.remove(CASE_NAME + '.clm2.h3.2017-01-01-00000.nc')
    
    #return average annual WTE
    return avgdata, coldata

In [ ]:
'''Functions'''
def nse(predictions, targets):
    return (1-(np.sum((targets-predictions)**2))/np.sum((targets-np.mean(targets))**2))

def rsquared(x, y):
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
    return r_value**2

def plotAvgHillslope(d, stream, spruce = None, clm = None, lab = None):
    fig, [ax1, ax2] = plt.subplots(1, 2, figsize = (10, 3), 
                           sharey = True)
    meas = np.array(stream['Flow_mms'])
    mod = np.array(d.QRUNOFF[d.time.dt.year == 2015]).reshape(365)
    
    #Plot 1 - Timeseries
    #Plot CLM hillslope - watershed level
    ax1.plot(mod, color = 'blue', alpha = 0.7, label = 'Hillslope Model')

    #Plot Marcell data
    ax1.plot(meas, color = 'silver',  label ='MEF')
    
    #Plot CLM-SPRUCE data
    if(spruce):
        sp = np.array(spruce.QRUNOFF[spruce.time.dt.year == 2015].isel(lndgrid=[1])).reshape(365)
        ax1.plot(sp, color = 'orange', alpha = 0.7, label = 'CLM-SPRUCE')
        
    if(clm):
        cl = np.array(clm.QRUNOFF[clm.time.dt.year == 2015]).reshape(365)
        ax1.plot(cl, color = 'teal', alpha = 0.7, label = 'CLM 5.0')

    #Plot spcifics
    ax1.set_xlabel('Date of the Year (2015)')
    ax1.set_ylabel('Total Liquid Runoff, modelled [mm/s]]')
    ax1.set_xlim(0, 365)
    ax1.set_ylim(0, 5e-4)
    
    #Plot 2 - Comparison
    ax2.scatter(meas, mod, color = 'blue', alpha = 0.5)
    ax2.scatter(meas, sp, color = 'orange', alpha = 0.5)
    ax2.axline([0,0], slope = 1, color = 'silver', zorder = -2)
    ax2.set_xlabel('Streamflow, measured [mm/s]')
    ax2.set_xlim(0, 5e-4)
    ax2.set_ylim(0, 5e-4)
    #ax2.set_aspect('equal')
    
    nse_plot = nse(mod, meas)
    nse_plot_spruce = nse(sp, meas)
    ax2.text(1.5e-4, 5e-5, r'$NSE_{hillslope}$: ' + str(round(nse_plot, 2)) + r', $NSE_{spruce}$: ' + str(round(nse_plot_spruce, 2)))
    r2_plot = rsquared(meas, mod)
    r2_plot_spruce = rsquared(meas, sp)
    ax2.text(1.5e-4, 1e-4, r'$R_{hillslope}^2$: ' + str(round(r2_plot, 2)) + r', $R_{spruce}^2$: ' + str(round(r2_plot_spruce, 2)))

    if(lab):        
        plt.suptitle('CLM Hillslope Model -- Watershed Scale, ' + lab)
    else:
        plt.suptitle('CLM Hillslope Model -- Watershed Scale')
    ax1.legend(bbox_to_anchor=(0.5, 1.4))
    plt.show()

In [ ]:
'''CLM UNCALIBRATED CASE - Lagg'''
#Import
file_dir = '/glade/scratch/marielj/archive/test-hillslope-mct-srof-lagg/lnd/hist/'
files = glob(file_dir + 'test-hillslope-mct-srof-lagg.clm2.h2.*.nc')
colfiles = glob(file_dir + 'test-hillslope-mct-srof-lagg.clm2.h3.*.nc')

data = xr.open_mfdataset(files, parallel = True)
coldata = xr.open_mfdataset(colfiles, parallel = True)

In [ ]:
#Plot Column Data
plotAvgHillslope(data, stream, SPRUCEuncalib)

In [ ]:
'''CLM MEASURED PARAMS'''
# h_pftndx - the pft associated with each column (currently set to 7-lagg, 10-bog, 7-uplands)
# h_bedrock - the depth to bedrock in each column (curently set to 8m-lagg, 8m-bog, 8m-uplands)
#Import
file_dir = '/glade/scratch/marielj/archive/test-hillslope-mct-srof-measparam/lnd/hist/'
files = glob(file_dir + 'test-hillslope-mct-srof-measparam.clm2.h2.*.nc')
colfiles = glob(file_dir + 'test-hillslope-mct-srof-measparam.clm2.h3.*.nc')

dataMeas = xr.open_mfdataset(files)
coldataMeas = xr.open_mfdataset(colfiles)

In [ ]:
#Plot Column Data
plotAvgHillslope(dataMeas, stream, SPRUCEuncalib)

In [ ]:
'''REMOVE OVERLAND FLOW'''
#FMAX = 0 using namelist options
case_name = 'hillslope_tuning_nospinup_FMAX_v7'
file_dir = '/glade/scratch/marielj/archive/hillslope_tuning_nospinup_FMAX_v7/lnd/hist/'
files = glob(file_dir + case_name + '.clm2.h2.*.nc')
colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

fmaxdata = xr.open_mfdataset(files)
fmaxcoldata = xr.open_mfdataset(colfiles)

In [ ]:
'''SURFACE file params'''
surf = xr.load_dataset('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_mineral_nofmax.nc')
#surf.FMAX[0] = 0
#surf.to_netcdf('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_mineral_nofmax.nc')

In [ ]:
plotAvgHillslope(fmaxdata, stream, SPRUCEuncalib)

In [ ]:
#FMAX = 0 using surface file options
file_dir = '/glade/scratch/marielj/archive/test-hillslope-mct-srof-fmax/lnd/hist/'
case_name = 'test-hillslope-mct-srof-fmax'
files = glob(file_dir + case_name + '.clm2.h2.*.nc')
colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

fmaxsurfdata = xr.open_mfdataset(files)
fmaxsurfcoldata = xr.open_mfdataset(colfiles)

In [ ]:
plotAvgHillslope(fmaxsurfdata, stream)

In [ ]:
#Mineral soils
file_dir = '/glade/scratch/marielj/archive/test-hillslope-mct-srof-mineral/lnd/hist/'
case_name = 'test-hillslope-mct-srof-mineral'
files = glob(file_dir + case_name + '.clm2.h2.*.nc')
colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

mineralsurfdata = xr.open_mfdataset(files)
mineralsurfcoldata = xr.open_mfdataset(colfiles)

In [ ]:
plotAvgHillslope(mineralsurfdata, stream, SPRUCEuncalib)

In [ ]:
#4 col original file
file_dir = '/glade/scratch/marielj/archive/test-hillslope-mct-srof/lnd/hist/'
case_name = 'test-hillslope-mct-srof'
files = glob(file_dir + case_name + '.clm2.h2.*.nc')
colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

colfoursurfdata = xr.open_mfdataset(files)
colfoursurfcoldata = xr.open_mfdataset(colfiles)

In [ ]:
plotAvgHillslope(colfoursurfdata, stream, SPRUCEuncalib)

## 1D Bayesian Parameter Search (FMAX)

In [ ]:
from clmmodtools import *

In [ ]:
#start with single parameter variation
def blackbox_clm(x):
    #change parameter in netcdf file
    target_surface_file = '/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_nofmax.nc'
    target_param = 'FMAX'
    
    change_surf_param(target_param, x, 0, target_surface_file)
    
    os.chdir(CASE_DIR)
    
    #Build case
    #pipe = subprocess.Popen(['qcmd', ' -- ./case.build'], stdout=subprocess.PIPE)
    
    #run case
    pipe = subprocess.Popen(['./case.submit'], stdout=subprocess.PIPE)
    #result = pipe.communicate()[0]
    #print(result)
    #print(CASE_NAME + " Run Complete")
    
    #time delay -- check if archived data exists, if not wait 5 more seconds
    SCRATCH_DIR = '/glade/scratch/marielj/archive/' + CASE_NAME + '/lnd/hist/'
    while(not os.path.exists(SCRATCH_DIR + CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')):
        time.sleep(5)
    
    #find stream data in scrath directory
    os.chdir(SCRATCH_DIR)
    
    #Open data h2 (daily but no column) data file
    dat = xr.load_dataset(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    mean_stream = -np.mean(dat.QRUNOFF.values)
    #remove data
    os.remove(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    
    #mean_wte = np.double(416.000) - mean_wtd
    
    #return average annual WTE
    return mean_stream

In [ ]:
def posterior(optimizer, x_obs, y_obs, grid):
    optimizer._gp.fit(x_obs, y_obs)

    mu, sigma = optimizer._gp.predict(grid, return_std=True)
    return mu, sigma

In [ ]:
def plot_gp_adapted(optimizer, x, param):
    fig = plt.figure(figsize=(8, 5))
    steps = len(optimizer.space)
    fig.suptitle(
        'Gaussian Process and Utility Function After {} Steps'.format(steps),
        fontdict={'size':30}
    )
    
    #Setup grid
    gs = gridspec.GridSpec(2, 1, height_ratios=[3, 1]) 
    axis = plt.subplot(gs[0])
    acq = plt.subplot(gs[1])
    
    #Observation points 
    x_obs = np.array([[res["params"][param]] for res in optimizer.res])
    y_obs = np.array([res["target"] for res in optimizer.res])
    
    #Interpolate the confidence interval using skilearn
    mu, sigma = posterior(optimizer, x_obs, y_obs, x)
    axis.plot(x_obs.flatten(), y_obs, 'D', markersize=8, label=u'Observations', color='r')
    axis.plot(x, mu, '--', color='k', label='Prediction')

    #Plot the Confidence interval using error from posterior simulation
    axis.fill(np.concatenate([x, x[::-1]]), 
              np.concatenate([mu - 1.9600 * sigma, (mu + 1.9600 * sigma)[::-1]]),
        alpha=.6, fc='c', ec='None', label='95% confidence interval')
    
    axis.set_ylim((None, None))
    axis.set_ylabel('Total Streamflow Runoff', fontdict={'size':20})
    axis.set_xlabel(param, fontdict={'size':20})
    
    utility_function = UtilityFunction(kind="ucb", kappa = 0.1)
    utility = utility_function.utility(x, optimizer._gp, 0)
    acq.plot(x, utility, label='Utility Function', color='purple')
    acq.plot(x[np.argmax(utility)], np.max(utility), '*', markersize=15, 
             label=u'Next Best Guess', markerfacecolor='gold', markeredgecolor='k', markeredgewidth=1)
    #acq.set_ylim((0, np.max(utility) + 0.5))
    acq.set_ylabel('Utility', fontdict={'size':20})
    acq.set_xlabel(param, fontdict={'size':20})
    
    axis.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    acq.legend(loc=2, bbox_to_anchor=(1.01, 1), borderaxespad=0.)
    
    plt.savefig('/glade/u/home/marielj/cesm-hillslope/calibration-runs/bayes-opt/bayesopt_' + param + '_1d_probing.pdf')

In [ ]:
# Setup case
CASE_NAME = 'test-hillslope-mct-srof-fmax'
CASE_DIR = '/glade/u/home/marielj/cesm-hillslope/' + CASE_NAME

In [ ]:
#Declare optimizer
clm_optimizer = BayesianOptimization(blackbox_clm, 
                                    {'x': (0, 0.1)}, #x is fmax
                                    random_state = 45729, 
                                    )
#Aquisition function
acquisition_function = UtilityFunction(kind="ucb", kappa=10)

In [ ]:
for p in np.linspace(0, 0.1, 5):
    clm_optimizer.probe(
        params={"x": p},
        lazy=True,
    )
    
clm_optimizer.maximize(init_points = 0, n_iter = 0)

In [ ]:
x_fmax = np.linspace(0, 0.1, 100).reshape(-1, 1)
plot_gp_adapted(clm_optimizer, x_fmax, 'x')

## 2D Bayesian Parameter Search (FMAX, Baseflow)

In [ ]:
def blackbox_NDclm(baseflow, fmax, sand, clay):
    #change parameter in netcdf file
    target_surface_file = '/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_mineral_nofmax.nc'
    target_param1 = 'baseflow_scalar'
    target_param2 = 'FMAX' 
    target_param3 = 'PCT_CLAY'
    target_param4 = 'PCT_SAND'
    
    #change fmax
    change_surf_param(target_param2, fmax, 0, target_surface_file)
    #Assign to the top part of the soil column (indices 0 through 7)
    for i in range(0, 8):
        change_surf_param(target_param3, clay, i, target_surface_file)
        change_surf_param(target_param4, sand, i, target_surface_file)
    
    os.chdir(CASE_DIR)
    
    #change baseflow_scalar
    change_nl_param(target_param1, baseflow)
    
    #run case
    pipe = subprocess.Popen(['qcmd', '-- ./case.submit'], stdout=subprocess.PIPE)
    #result = pipe.communicate()[0]
    #print(result)
    #print(CASE_NAME + " Run Complete")
    
    #time delay -- check if archived data exists, if not wait 5 more seconds
    SCRATCH_DIR = '/glade/scratch/marielj/archive/' + CASE_NAME + '/lnd/hist/'
    while(not os.path.exists(SCRATCH_DIR + CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')):
        time.sleep(5)
    
    #find WTE data in scracth directory
    os.chdir(SCRATCH_DIR)
    
    #Open data h1 data file
    dat = xr.load_dataset(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    mod = dat.QRUNOFF.values.reshape(365)
    meas = np.array(stream['Flow_mms'])
    
    #Compute correlation
    r2_plot = rsquared(meas, mod)
    
    #remove data
    os.remove(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    
    #return average annual WTE
    return r2_plot

In [ ]:
def blackbox_clm2D(baseflow, fmax):
    #change parameter in netcdf file
    target_surface_file = '/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_nofmax.nc'
    target_param1 = 'baseflow_scalar'
    target_param2 = 'FMAX' 
    
    #change fmax
    change_surf_param(target_param2, fmax, 0, target_surface_file)
    
    os.chdir(CASE_DIR)
    
    #change baseflow_scalar
    change_nl_param(target_param1, baseflow)
    
    #run case
    pipe = subprocess.Popen(['qcmd', '-- ./case.submit'], stdout=subprocess.PIPE)
    #result = pipe.communicate()[0]
    #print(result)
    #print(CASE_NAME + " Run Complete")
    
    #time delay -- check if archived data exists, if not wait 5 more seconds
    SCRATCH_DIR = '/glade/scratch/marielj/archive/' + CASE_NAME + '/lnd/hist/'
    while(not os.path.exists(SCRATCH_DIR + CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')):
        time.sleep(5)
    
    #find WTE data in scracth directory
    os.chdir(SCRATCH_DIR)
    
    #Open data h1 data file
    dat = xr.load_dataset(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    mod = dat.QRUNOFF.values.reshape(365)
    meas = np.array(stream['Flow_mms'])
    
    #Compute correlation
    r2_plot = rsquared(meas, mod)
    
    #remove data
    os.remove(CASE_NAME + '.clm2.h2.2015-01-01-00000.nc')
    
    #return average annual WTE
    return r2_plot

In [ ]:
def plot_gp_2D(optimizer, aq, it, param1, param2, param1_min, param1_max, param2_min, param2_max):
    #Setup 
    #Breakdown Data
    max_ = optimizer.max
    res = optimizer.res[:it]
    x_ = np.array([r["params"][param1] for r in res])
    y_ = np.array([r["params"][param2] for r in res])
    z_ = np.array([r["target"] for r in res])
    points = pd.DataFrame({'baseflow' : x_,
              'fmax': y_})

    #Model Results
    ser = pd.Series(z_,
                      index=[y_, x_])
    Z = ser.unstack().fillna(np.nan)

    #Range space
    x1 = np.linspace(param1_min, param1_max, 100).reshape(-1, 1)
    x2 = np.linspace(param2_min, param2_max, 100).reshape(-1, 1)
    xmesh, ymesh = np.meshgrid(x1, x2)
    xy = np.array(list(zip(xmesh.reshape(10000, 1), ymesh.reshape(10000, 1)))).reshape(10000,2)

    #Utility Function
    util = aq.utility(xy, optimizer._gp, 0)
    
    #Plot
    fig, axs = plt.subplots(1, 1, constrained_layout=True, figsize=(6,6))

    #Axis 1: Function Estimate
    grid_z0 = griddata(points, z_, xy, method='nearest').reshape(100,100)
    mesh1 = axs.pcolormesh(xmesh, ymesh,  grid_z0, cmap=plt.cm.coolwarm)
    axs.scatter(x_, y_, c='white', s=80, edgecolors='black')
    axs.scatter(x_, y_, c='red', s=80, edgecolors='black')
    axs.scatter(max_["params"][param1], max_["params"][param2], s=80, c='yellow', edgecolors='black')

    axs.set_title('Function Estimate')
    axs.set_xlabel(param1)
    axs.set_ylabel(param2)

    axs.set_xlim(param1_min, param1_max)
    axs.set_ylim(param2_min, param2_max)

    fig.colorbar(mesh1, location = 'bottom', label = r"Streamflow $R^2$")
    plt.savefig('/glade/u/home/marielj/cesm-hillslope/figures/bayesopt/bayesopt_' + param1 + '_' + param2 + '_2d.pdf')

    plt.show()

In [ ]:
#Declare optimizer
clm_optimizer2D = BayesianOptimization(f = blackbox_clm2D, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4)}, 
                                    random_state = 4889, 
                                    verbose = 0
                                    )
#Aquisition function
acquisition_function2D = UtilityFunction(kind="ucb", kappa=10000)

#Optimize
clm_optimizer2D.maximize(init_points = 0, n_iter = 5, aquisition_function = acquisition_function2D, allow_duplicate_points=True)

In [ ]:
plot_gp_2D(clm_optimizer2D, acquisition_function2D, 6, 'baseflow', 'fmax', 0, 1, 0, 0.4)

## Plot Bayes Opt Runscript

### Explorative

In [ ]:
#Load data from json
from bayes_opt.util import load_logs

new_optimizer_explore = BayesianOptimization(f = blackbox_clm2D, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4)}, 
                                    random_state = 44324, 
                                    verbose = 0
                                    )
acquisition_function = UtilityFunction(kind="ucb", kappa=10e2)

load_logs(new_optimizer_explore, logs=["/glade/u/home/marielj/cesm-hillslope/hillslope_full_log.json"]);

In [ ]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer_explore.space)))

In [ ]:
plot_gp_2D(new_optimizer_explore, acquisition_function, 71, 'baseflow', 'fmax', 0, 1, 0, 0.4)

In [ ]:
#Pull out correlation values
y_explore = [new_optimizer_explore.res[i]['target'] for i in range(0, 71)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 3))
sns.regplot(x = np.arange(1, 72), y = y_explore, ax = ax)
ax.set_xlabel('Iteration')
ax.set_ylabel(r'$R^2$')

In [ ]:
new_optimizer_explore.res[41]

### Explotative

In [ ]:
#Load data from json
from bayes_opt.util import load_logs

new_optimizer_exploit = BayesianOptimization(f = blackbox_clm2D, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4)}, 
                                    random_state = 44324, 
                                    verbose = 0
                                    )
acquisition_function = UtilityFunction(kind="ucb", kappa=0.1)

load_logs(new_optimizer_exploit, logs=["/glade/u/home/marielj/cesm-hillslope/hillslope_full_log_exploitative.json"]);

In [ ]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer_exploit.space)))

In [ ]:
plot_gp_2D(new_optimizer_exploit, acquisition_function, 35, 'baseflow', 'fmax', 0, 1, 0, 0.4)

In [ ]:
#Pull out correlation values
y_exploit = [new_optimizer_exploit.res[i]['target'] for i in range(0, 35)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 3))
sns.regplot(x = np.arange(1, 36), y = y_exploit, ax = ax)
ax.set_xlabel('Iteration')
ax.set_ylabel(r'$R^2$')

In [ ]:
new_optimizer_exploit.res[34]

## Explorative on Mineral Soil

In [ ]:
#Load data from json
new_optimizer_mineral = BayesianOptimization(f = blackbox_clm2D, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4)}, 
                                    random_state = 44324, 
                                    verbose = 0
                                    )
acquisition_function = UtilityFunction(kind="ucb", kappa=10e2)

load_logs(new_optimizer_mineral, logs=["/glade/u/home/marielj/cesm-hillslope/hillslope_full_mineral_log.json"]);

In [ ]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer_mineral.space)))

In [ ]:
plot_gp_2D(new_optimizer_mineral, acquisition_function, 36, 'baseflow', 'fmax', 0, 1, 0, 0.4)

In [ ]:
#Pull out correlation values
y_mineral = [new_optimizer_mineral.res[i]['target'] for i in range(0, 35)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 3))
sns.regplot(x = np.arange(1, 36), y = y_mineral, ax = ax)
ax.set_xlabel('Iteration')

In [ ]:
new_optimizer_mineral.max

## Mineral Soil with Soil Parameters

In [ ]:
new_optimizer_mineral4D = BayesianOptimization(f = blackbox_NDclm, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4), 'sand' : (0, 1), 'clay' : (0, 1)}, 
                                    random_state = 44738, 
                                    verbose = 0
                                    )

load_logs(new_optimizer_mineral4D, logs=["/glade/u/home/marielj/cesm-hillslope/hillslope_full_mineral4D_log.json"]);

In [ ]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer_mineral4D.space)))

In [ ]:
#Pull out correlation values
y_mineral4D = [new_optimizer_mineral4D.res[i]['target'] for i in range(0, 23)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 3))
sns.regplot(x = np.arange(1, 24), y = y_mineral4D, ax = ax)
ax.set_xlabel('Iteration')

## Calibration to WTE

In [ ]:
from bayes_opt.util import load_logs

def blackbox_clm_wte(baseflow, fmax):
    #change parameter in netcdf file
    target_surface_file = '/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_nofmax.nc'
    target_param1 = 'baseflow_scalar'
    target_param2 = 'FMAX' 
    
    #change fmax
    change_surf_param(target_param2, fmax, 0, target_surface_file)
    
    os.chdir(CASE_DIR)
    
    #change baseflow_scalar
    change_nl_param(target_param1, baseflow)
    
    #run case
    pipe = subprocess.Popen(['qcmd', '-- ./case.submit'], stdout=subprocess.PIPE)
    #result = pipe.communicate()[0]
    #print(result)
    #print(CASE_NAME + " Run Complete")
    
    #time delay -- check if archived data exists, if not wait 5 more seconds
    #Have to check for the column specific run here for proper bog wte calibration
    SCRATCH_DIR = '/glade/scratch/marielj/archive/' + CASE_NAME + '/lnd/hist/'
    while(not os.path.exists(SCRATCH_DIR + CASE_NAME + '.clm2.h3.2015-01-01-00000.nc')):
        time.sleep(5)
    
    #find WTE data in scracth directory
    os.chdir(SCRATCH_DIR)
    
    #Open data h1 data file
    dat = xr.load_dataset(CASE_NAME + '.clm2.h3.2015-01-01-00000.nc')
    mod = -np.array(dat.sel(column = 1).ZWT).reshape(365) #take SPECIFICALLY the bog water table
    meas = np.array(wte.WTD)
    
    #Compute correlation
    r2_plot = rsquared(meas, mod)
    
    #remove data
    os.remove(CASE_NAME + '.clm2.h3.2015-01-01-00000.nc')
    
    #return average annual WTE
    return r2_plot

acquisition_function = UtilityFunction(kind = "ucb", kappa = 10e2)

new_optimizer_wte = BayesianOptimization(f = blackbox_clm_wte, 
                                    pbounds = {'baseflow': (0,1), 'fmax': (0, 0.4)}, 
                                    random_state = 44738, 
                                    verbose = 0
                                    )

load_logs(new_optimizer_wte, logs=["/glade/u/home/marielj/cesm-hillslope/hillslope_full_wte_log.json"]);

In [ ]:
print("New optimizer is now aware of {} points.".format(len(new_optimizer_wte.space)))

In [ ]:
#Pull out correlation values
y_wte = [new_optimizer_wte.res[i]['target'] for i in range(0, 35)]

In [ ]:
fig, ax = plt.subplots(1, 1, figsize = (5, 3))
sns.regplot(x = np.arange(1, 36), y = y_wte, ax = ax)
ax.set_xlabel('Iteration')

In [ ]:
plot_gp_2D(new_optimizer_wte, acquisition_function, 36, 'baseflow', 'fmax', 0, 1, 0, 0.4)

## Select Bayes Opt cases

In [ ]:
#baseflow = 0.076, fmax = 0.155, organic soil
surf = xr.load_dataset('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_nofmax.nc')
surf.FMAX[0] = 0.11619085548362439
surf.to_netcdf('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_fmax0.116.nc')

In [ ]:
case_name = 'bayes-opt-fmax0.116-baseflow0.74'
file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
selectfiles = glob(file_dir + case_name + '.clm2.h2.*.nc')
selectcolfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

selectdata = xr.open_mfdataset(selectfiles)
selectcoldata = xr.open_mfdataset(selectcolfiles)

In [ ]:
plotAvgHillslope(selectdata, stream, SPRUCEuncalib)

In [ ]:
selectcoldata.QINFL[selectcoldata.time.dt.year == 2015].plot(x = 'time', hue = 'column')

In [ ]:
#baseflow = 0.445, fmax = 0.107, organic soil
surf = xr.load_dataset('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_nofmax.nc')
surf.FMAX[0] = 0.10798314556601483
surf.to_netcdf('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_fmax0.108.nc')

In [ ]:
case_name = 'bayes-opt-fmax0.108-baseflow0.445'
file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
selectfiles2 = glob(file_dir + case_name + '.clm2.h2.*.nc')
selectcolfiles2 = glob(file_dir + case_name + '.clm2.h3.*.nc')

selectdata2 = xr.open_mfdataset(selectfiles2)
selectcoldata2 = xr.open_mfdataset(selectcolfiles2)

In [ ]:
plotAvgHillslope(selectdata2, stream, SPRUCEuncalib)

In [ ]:
#baseflow = 0.818, fmax = 0.236, mineral soil
surf = xr.load_dataset('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_mineral_nofmax.nc')
surf.FMAX[0] = 0.23639743403996652
surf.to_netcdf('/glade/work/marielj/inputdata/lnd/clm2/surfdata_map/hillslope/surfdata_1x1pt_US-MBP_hist_16pfts_Irrig_CMIP6_simyr2000_HAND_3_col_hillslope_lagg_pft_soildepth_mineral_fmax0.236.nc')

In [ ]:
case_name = 'bayes-opt-mineral-fmax0.236-baseflow0.818'
file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
selectfiles2 = glob(file_dir + case_name + '.clm2.h2.*.nc')
selectcolfiles2 = glob(file_dir + case_name + '.clm2.h3.*.nc')

selectdata3 = xr.open_mfdataset(selectfiles2)
selectcoldata3 = xr.open_mfdataset(selectcolfiles2)

In [ ]:
plotAvgHillslope(selectdata3, stream, SPRUCEuncalib)

In [ ]:
selectcoldata3.QINFL[selectcoldata3.time.dt.year == 2015].plot(x = 'time', hue = 'column')

## OLD

In [ ]:
'''Plot FMAX calibration'''
# FMAX - Maximum value of surface runoff due to saturation excess
fmax = [0.0001, 0.001, 0.005, 0.01, 0.05, 0.075, 0.1, 0]

for i in range(0,len(fmax)):
    case_name = 'hillslope_tuning_nospinup_FMAX_v' + str(i) 
    #Import
    file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
    files = glob(file_dir + case_name + '.clm2.h2.*.nc')
    colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

    data = xr.open_mfdataset(files)
    coldata = xr.open_mfdataset(colfiles)
    
    #Plot
    plotAvgHillslope(data, stream, lab = str(fmax[i]))

In [ ]:
'''Plot BASEFLOW calibration'''
# BASEFLOW - Scaling factor for baseflow/drainage from the gridcell
baseflow = [10e-8, 10e-7, 10e-6, 10e-4, 10e-2, 1, 2]

for i in range(0,len(baseflow)):
    case_name = 'hillslope_tuning_nospinup_BASEFLOW_v' + str(i) 
    #Import
    file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
    files = glob(file_dir + case_name + '.clm2.h2.*.nc')
    colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

    data = xr.open_mfdataset(files)
    coldata = xr.open_mfdataset(colfiles)
    
    #Plot
    plotAvgHillslope(data, stream, lab = str(baseflow[i]))

In [ ]:
'''Plot PCT_CLAY calibration'''
clay = [0, 5, 10, 20, 30]

for i in range(0,len(clay)):
    case_name = 'hillslope_tuning_nospinup_PCT_CLAY_v' + str(i) 
    #Import
    file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
    files = glob(file_dir + case_name + '.clm2.h2.*.nc')
    colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

    data = xr.open_mfdataset(files)
    coldata = xr.open_mfdataset(colfiles)
    
    #Plot
    plotAvgHillslope(data, stream, lab = str(clay[i]))

In [ ]:
'''Plot PCT_SAND calibration'''
sand = [0, 5, 10, 20, 30]

for i in range(0,len(sand)):
    case_name = 'hillslope_tuning_nospinup_PCT_SAND_v' + str(i) 
    #Import
    file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
    files = glob(file_dir + case_name + '.clm2.h2.*.nc')
    colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

    data = xr.open_mfdataset(files)
    coldata = xr.open_mfdataset(colfiles)
    
    #Plot
    plotAvgHillslope(data, stream, lab = str(sand[i]))

In [ ]:
'''PARAMETERS from PARAM file'''
params = xr.load_dataset('/glade/u/home/marielj/cesm-hillslope/calibration-runs/params/clm50_params.c211112.nc')

In [ ]:
params

In [ ]:
'''SLATOP - Arctic Grass'''
slatop = [0.004, 0.008, 0.016, 0.032, 0.04]

for i in range(0,len(slatop)):
    case_name = 'hillslope_tuning_nospinup_SLATOP_v' + str(i) 
    #Import
    file_dir = '/glade/scratch/marielj/archive/' + case_name + '/lnd/hist/'
    files = glob(file_dir + case_name + '.clm2.h2.*.nc')
    colfiles = glob(file_dir + case_name + '.clm2.h3.*.nc')

    data = xr.open_mfdataset(files)
    coldata = xr.open_mfdataset(colfiles)
    
    #Plot
    plotAvgHillslope(data, stream, lab = str(slatop[i]))